Read in all the packages

In [1]:
import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
import random

# import geemap
# Authenticate GEE
# ee.Authenticate()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)
# Initialize GEE with high-volume end-point
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Initialize()


In [2]:
import geemap
from google.cloud import storage
from google.cloud import client

Read in assets

In [3]:
sent_2A = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") #sentinel 2
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') #cloud masking for sentinel
lfdb = ee.FeatureCollection("users/spotter/fire_cnn/ann_w_id") #anna polygons 

#need to add ids to annas polygons
# Create a sequence of numbers from 1 to the number of features
# indexes = lfdb.aggregate_array('system:index')
# ids = list(range(1, indexes.size().getInfo() + 1))
# idByIndex = ee.Dictionary.fromLists(indexes, ids)

# # Map over the collection and set the 'ID' property
# lfdb = lfdb.map(lambda feature: feature.set('ID', idByIndex.get(feature.get('system:index'))))



Mask clouds Sentinel

In [4]:

#probability of clouds
MAX_CLOUD_PROBABILITY = 50

def sent_maskcloud(image):
    
    
    image = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'], ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])# rename bands to match landsat
  
    image =  image.toShort()
    
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    
    image = image.updateMask(isNotCloud)

    #reproject 30m but remember b1, b2 and b3 are 10 and the rest are 20
    image1 = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4'])
    image2 = image.select(['SR_B5', 'SR_B7'])

    
    image1 = image1.reproject(
    crs = image1.projection().crs(),
    scale = 30) #resample for landsat
    
    
    image2 = image2.reproject(
    crs = image2.projection().crs(),
    scale = 30) #resample for landsat
    
    image = image1.addBands(image2)
    
    return image 

#Join S2 SR with cloud probability dataset to add cloud mask.
s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
    
  primary=sent_2A,
  secondary=s2Clouds,
  condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

#apply cloud masking
sent_2A = ee.ImageCollection(s2SrWithCloudMask).map(sent_maskcloud)


Mask clouds


In [5]:
def mask_s2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0) \
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    return image.updateMask(mask).divide(10000) #dividing is the scale factor application

Coefficients from Logan Berner to apply correction factors

Authenticate my google bucket, not sure if this is needed anymore

In [8]:
os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"
storage_client = storage.Client.from_service_account_json("/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")

os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)

There are going to be issues later with different band types, so cast them all to float

In [9]:
def to_float(image):

    b1 = image.select('SR_B1').cast({'SR_B1':'float'}) #0
    b2 = image.select('SR_B2').cast({'SR_B2':'float'}) #1
    b3 = image.select('SR_B3').cast({'SR_B3':'float'}) #2
    b4 = image.select('SR_B4').cast({'SR_B4':'float'}) #3
    b5 = image.select('SR_B5').cast({'SR_B5':'float'}) #4
    b6 = image.select('SR_B7').cast({'SR_B7':'float'}) #5

    image = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6)

    return image

Function to get the HLS post fire data for a specific time frame.  ONly need to specify post fire dates here.  I will download a variety using different months to test.  Note HLS is only 2017-present.  It seems really only 2019 though.

In [ ]:
def get_imagery(start, end, geometry):
    
    #hls image collection, filter date, geometry, cloud masking, take median
    hls = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start, end).filterBounds(geometry).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).map(mask_s2clouds).median()
    
    #resample blue green red to 20m
    bgr = hls.select(['B2', 'B3', 'B4'])
    
    #select nir, swir1, swir2, already at 20m
    ns = hls.select(['B8', 'B11', 'B12'])
    
    bgr= bgr.reproject(
        crs = image1.projection().crs(),
        scale = 20) #resample for landsat
    
    
    image = bgr.addBands(ns)
    
    #return the image
    return image
    
    
    

In [19]:

#get all the ids within the lfdb shapefile
all_ids = ee.List(lfdb.distinct(["ID"]).aggregate_array("ID"))
all_ids = all_ids.getInfo()

# all_ids = all_ids[0:2]

pre_months = ['-04-01', '-05-01', '-06-01', '-07-01', '-08-01', '-09-01']
end_months = ['-05-01', '-06-01', '-07-01', '-08-01', '-09-01', '-10-01']

all_months = dict(zip(pre_months, end_months))

#these ids have a mix of grassland (early season) and siberia (late season) burns.
all_ids = 
#loop through each fire polygon
for i in all_ids:

        
    #get the fire polygon of interest
    sub_shape = lfdb.filter(ee.Filter.eq("ID", i))

    #get all other fire ids that are not this one
    not_fires = lfdb.filter(ee.Filter.neq("ID", i))


    #first get the bounding box of the fire
    bbox = sub_shape.geometry().bounds()


    #offset the bounding box by a random number
    all_rands = [0.00, 0.02, -0.02]

    rand1 = random.sample(all_rands, 1)[0]
    rand2 = random.sample(all_rands, 1)[0]

    #offset applied
    proj = ee.Projection("EPSG:4326").translate(rand1, rand2)

    #for the bounding box apply the randomly selected offset
    final_buffer = ee.Geometry.Polygon(bbox.coordinates(), proj).transform(proj)

    #this is a bit of a hack but we have two different bounding box sizes because when we export we need to use some additonal area to avoid cuttoffs
    final_buffer2 = final_buffer.buffer(distance= 2000).bounds()

    final_buffer = final_buffer.buffer(distance= 40000)#.bounds().transform(proj='EPSG:3413', maxError=1)

    #get the year of this fire
    this_year = ee.Number(sub_shape.aggregate_array('Year').get(0))

    year = this_year.getInfo() 

#     pre_start = ee.Date.fromYMD(this_year.subtract(1), 6, 1)
#     pre_end = ee.Date.fromYMD(this_year.subtract(1), 8, 31)
#     post_start = pre_start.advance(2, 'year')
#     post_end = pre_end.advance(2, 'year')

#     #just getting some date info here to ensure pre fire is one  year before and post fire is one year after the fire year of interest
#     startYear = pre_start.get('year')

#     #convert to client side
#     startYear = startYear.getInfo()  # local string
#     endYear = str(int(startYear) + 2)
#     startYear = str(startYear)

        
     #loop through all months 
    for m1, m2 in all_months.items():
        
        m1_name = m1.replace('-', '_')[1:]
        m2_name = m2.replace('-', '_')
        #name of output file
        fname = f"{m1_name}{m2_name}_all_bands_median_{i}.tif"

        #check if file exists on my bucket, if it does skip
        stats = storage.Blob(bucket=bucket, name=fname).exists(storage_client)

        if stats == False:

            #get imagary dates

            start = str(year) + m1
            end = str(year) + m2



            #apply the function to get the pre_fire image and post_fire image
            this_image =  get_imagery(start, end final_buffer)
            
            #ensure this image has data
            
#             #return the pre and post fire input imagery lists
#             pre_input = all_imagery[0]
#             post_input = all_imagery[1]

#             #if the lists each are larger than 1 we have imagery
#             if (len(pre_input) >0) and (len(post_input) > 0):

#                 #take the median of the image collections
#                 pre_input = ee.ImageCollection(pre_input)
#                 post_input = ee.ImageCollection(post_input)

#                 #get median of images
#                 pre_input = pre_input.median()
#                 post_input= post_input.median()

#                 #difference the bands
#                 raw_bands = pre_input.subtract(post_input).multiply(1000)

#                 b1 = raw_bands.select('SR_B1').cast({'SR_B1':'short'})
#                 b2 = raw_bands.select('SR_B2').cast({'SR_B2':'short'})
#                 b3 = raw_bands.select('SR_B3').cast({'SR_B3':'short'})
#                 b4 = raw_bands.select('SR_B4').cast({'SR_B4':'short'})
#                 b5 = raw_bands.select('SR_B5').cast({'SR_B5':'short'})
#                 b6 = raw_bands.select('SR_B7').cast({'SR_B7':'short'})
#                 b7 = raw_bands.select('NBR').cast({'NBR':'short'})
#                 b8 = raw_bands.select('NDVI').cast({'NDVI':'short'})
#                 b9 = raw_bands.select('NDII').cast({'NDII':'short'})

#                 #combine all the bands
#                 raw_bands = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9)
#                 # raw_bands = b7.addBands(b8).addBands(b9)

#                 all_months_images.append(raw_bands)


#         #append to all_days vi
#         raw_bands = ee.ImageCollection(all_months_images).reduce(ee.Reducer.percentile([85]))
        
#         b1 = raw_bands.select('SR_B1_p85').cast({'SR_B1_p85':'short'}) #0
#         b2 = raw_bands.select('SR_B2_p85').cast({'SR_B2_p85':'short'}) #1
#         b3 = raw_bands.select('SR_B3_p85').cast({'SR_B3_p85':'short'}) #2
#         b4 = raw_bands.select('SR_B4_p85').cast({'SR_B4_p85':'short'}) #3
#         b5 = raw_bands.select('SR_B5_p85').cast({'SR_B5_p85':'short'}) #4
#         b6 = raw_bands.select('SR_B7_p85').cast({'SR_B7_p85':'short'}) #5
#         # b7 = raw_bands.select('NBR_p85').cast({'NBR_p85':'short'}) #band 6 is dnbr is numpy
#         # b8 = raw_bands.select('NDVI_p85').cast({'NDVI_p85':'short'}) #7
#         # b9 = raw_bands.select('NDII_p85').cast({'NDII_p85':'short'}) #8


#         #if using all bands
#         raw_bands = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9)

#         # raw_bands = b7.addBands(b8).addBands(b9)


#         raw_bands = raw_bands.clip(final_buffer)
        
#         #we need to see which image ids from the entire lfdb are already included in the buffer
#         lfdb_filtered_orig = lfdb.filterBounds(final_buffer)

#         #ensure all fires are within the actual year of interest (this_year) and two years prior, otherwise ignore, this is to ensure we don't have nearby fires from previous years
#         first_year =  int(startYear) + 1
#         second_year =  int(startYear)
#         third_year =  int(startYear) - 1
#         fourth_year = int(startYear) + 2

#         lfdb_filtered = lfdb_filtered_orig.filter(ee.Filter.eq("Year", year))

#         bad_filtered = lfdb_filtered_orig.filter(ee.Filter.Or(ee.Filter.eq("Year", second_year), ee.Filter.eq("Year", third_year), ee.Filter.eq("Year", fourth_year)))


#         #get ids which are in image
#         all_ids_new = ee.List(lfdb_filtered.distinct(["ID"]).aggregate_array("ID")).getInfo()


#         #remove ids from all dates which we do not need anymore
#         all_ids2 = [i for i in all_ids if i not in all_ids_new]

#         #area we have good fires
#         fire_rast = lfdb_filtered.reduceToImage(properties= ['ID'], reducer = ee.Reducer.first())

#         #areas we have fires from other years or nearby we don't want to use
#         bad_fire_rast = bad_filtered.reduceToImage(properties= ['ID'], reducer = ee.Reducer.first())

#         #change values to 1 for fire of interest
#         fire_rast = fire_rast.where(fire_rast.gt(0), 1)

#         #change values for bad fire raster to 1 as well
#         bad_fire_rast = bad_fire_rast.where(bad_fire_rast.gt(0), 1)

#         #if the fires overlap we want to keep those locations
#         bad_fire_rast = bad_fire_rast.where(bad_fire_rast.eq(1).And(fire_rast.eq(1)), 2).unmask(-999)

#         #rename to y for the fire raster
#         fire_rast = fire_rast.rename(['y'])

#         #copy the first values of raw_bands
#         y = raw_bands.select(['NBR_p85'], ['y'])

#         #turn all values of y to 0
#         y  = y.where(y.gt(-10000), 0)

#         #turn values to 1 where fire_rast is 1
#         y  = y.where(fire_rast.eq(1), 1)

#         # #we need to ensure all bands are shorts 
#         # b1 = raw_bands.select('SR_B1').cast({'SR_B1':'short'}) 
#         # b2 = raw_bands.select('SR_B2').cast({'SR_B2':'short'}) 
#         # b3 = raw_bands.select('SR_B3').cast({'SR_B3':'short'}) 
#         # b4 = raw_bands.select('SR_B4').cast({'SR_B4':'short'}) 
#         # b5 = raw_bands.select('SR_B5').cast({'SR_B5':'short'}) 
#         # b6 = raw_bands.select('SR_B7').cast({'SR_B7':'short'}) 
#         # b7 = raw_bands.select('NBR').cast({'NBR':'short'}) 
#         # b8 = raw_bands.select('NDVI').cast({'NDVI':'short'}) 
#         # b9 = raw_bands.select('NDII').cast({'NDII':'short'}) 
#         b10 = y.select('y').cast({'y':'short'})

#         #combine all the bands for predictors
#         # raw_bands = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9)
#         # raw_bands = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9)

#         #for areas where there are nearby fires or fires in previous years we set those to 0
#         raw_bands = raw_bands.updateMask(bad_fire_rast.neq(1))

#         #add in the target variable
#         raw_bands = raw_bands.addBands(b10)

#         #start download
#         print(f"Downloading {fname}")


#         #export image to my cloud storage
#         task = ee.batch.Export.image.toCloudStorage(
#                                 image = raw_bands.toShort(),
#                                 region=final_buffer2, 
#                                 description='all_bands_median_' + str(i),
#                                 scale=30,
#                                 crs='EPSG:3413',
#                                 maxPixels=1e13,
#                                 bucket = 'smp-scratch')

#         task.start()
        
        
        
        
        
        




ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
't'

In [18]:
# Use reduceRegion to calculate the minimum and maximum values in the specified ROI
stats = raw_bands.select('NBR_p85').reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=final_buffer,
    scale=30,  # Set the scale according to the resolution of the image
    maxPixels=1e9  # Adjust maxPixels as needed
)

# Get the minimum and maximum values
min_value = stats.getNumber('NBR_p85_min')
max_value = stats.getNumber('NBR_p85_max')

# Print the results
print('Minimum Value:', min_value.getInfo())
print('Maximum Value:', max_value.getInfo())

Minimum Value: -953
Maximum Value: 1385
